In [1]:
import requests
import pandas as pd
import time
import numpy as np
import ast
from datetime import (datetime, timedelta, date)
import json

# import oandapyV20
# import oandapyV20.endpoints.instruments as instruments
# import oandapyV20.endpoints.orders as orders
# from oandapyV20.contrib.requests import (MarketOrderRequest, TrailingStopLossDetails)

# from ta import trend
# from ta import momentum

from oanda_forex_utilities import *

In [2]:
account_details = get_oanda_account_credentials("/Users/nguyenhoangnam/Documents/Personal Documents/oanda-forex/Forex",
                                          "demo_account_primary")
account_id = account_details["account_id"]
access_token = account_details["access_token"]

In [3]:
account_id

'101-003-18483320-001'

In [4]:
access_token

'87a216bb7b0e640e4af97a43499b58b9-4803472d0b66dd35b5d7541f3105caf5'

In [ ]:
extraction = False
while extraction == False:
#     time.sleep(0.5)
    time_now = datetime.utcnow()
    if (time_now.minute % 5 == 0) & (time_now.second == 0):
        print(time_now)
        extraction = True
time.sleep(5)
state = True
activity_dict = {}

while state == True:
    df = get_candle_data(account_id, access_token,"EUR_USD", "M5", 500)
    df_with_indicators = get_macd_indicator(df)
    df_with_indicators = get_sma_indicator(df_with_indicators)
    df_with_indicators = get_rsi_indicator(df_with_indicators)
    df_with_indicators_complete = df_with_indicators[df_with_indicators["complete"] == True]

    if ((((df_with_indicators_complete["macd_signal_diff"].iloc[-1] < 0) and
         (df_with_indicators_complete["macd_signal_diff"].iloc[-2] < 0) and
         (df_with_indicators_complete["macd_signal_diff"].iloc[-3] >= 0))|
        
        ((df_with_indicators_complete["macd_signal_diff"].iloc[-1] < -0.00005) and
         (df_with_indicators_complete["macd_signal_diff"].iloc[-2] >= 0))) and
        
        (df_with_indicators_complete["rsi_14"].iloc[-1] >= 30) and
        (get_position_details(account_id, access_token, "EUR_USD")["short"] == "0")):
        decision = "SELL"
        print(df_with_indicators_complete["time"].iloc[-1], "SELL")
        
        create_market_order_with_trailing_sl_only(account_id, access_token,"EUR_USD", "SELL", 5000, 10)

    elif ((((df_with_indicators_complete["macd_signal_diff"].iloc[-1] > 0) and
         (df_with_indicators_complete["macd_signal_diff"].iloc[-2] > 0) and
         (df_with_indicators_complete["macd_signal_diff"].iloc[-3] <= 0))|
        
        ((df_with_indicators_complete["macd_signal_diff"].iloc[-1] > 0.00005) and
         (df_with_indicators_complete["macd_signal_diff"].iloc[-2] <= 0))) and
        
        (df_with_indicators_complete["rsi_14"].iloc[-1] <= 70) and
        (get_position_details(account_id, access_token, "EUR_USD")["long"] == "0")):
        decision = "BUY"
        print(df_with_indicators_complete["time"].iloc[-1], "BUY")

        create_market_order_with_trailing_sl_only(account_id, access_token,"EUR_USD", "BUY", 5000, 10)
    else:
        print(df_with_indicators_complete["time"].iloc[-1], "Nothing happens")
        decision = "NOTHING HAPPENS"
    activity_dict[df_with_indicators_complete["time"].iloc[-1]] = decision
    time.sleep(300)

In [6]:
create_market_order_with_trailing_sl_only(account_id, access_token,"GBP_USD", "BUY", 3500, 20)